<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=755ffd6f5386884b7258383affcf1be0b3ceb0a6abc614e2110cf1cb96e1c06d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-15 12:25:58
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.41 C
-------------------
Today PnL: 29.66 K (0.21%)
Current PnL: -26.17 L (-16.91%)
CY Booked + Current PnL: -11.88 L (-7.67%)
-------------------
Total profit:  1.42 L
Total loss:  -27.59 L
-------------------
Total Booked + Current PnL: 15.10 L (11.81%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.14%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.81 L (64.46%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.04,-16.77,20.26,0.09,16994.0,-16898.0,83879.0,94.66,42.0,M-SC,2.36,253.0,-0.99,0.60,10.20,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.79,-11.66,13.22,0.02,20386.0,-20353.0,154206.0,134.93,54.0,M-SC,15.37,234.0,-1.00,1.10,34.13,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.74,-0.35,22.48,22.05,21948.0,-345.0,97635.0,-14.18,60.0,H-SC,7.19,164.0,-0.02,0.70,41.92,XR,ATH,FINANCE
82,VOLTAS,1530.00,0.19,7.94,10.89,19.69,22539.0,15228.0,206970.0,-2.60,55.0,X-MC,2.55,78.0,0.68,1.48,15.28,XY25,NTT,AC
43,ITC,452.00,0.47,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.69,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJOOENG,143.10,10.97,-31.29,103.18,39.61,72670.0,-32070.0,70430.0,-53.53,44.0,H-SC,27.90,135.0,-0.44,0.50,10.97,AR,ATH,MISC
18,COFFEEDAY,80.00,9.64,-39.60,123.96,35.27,85014.0,-44967.0,68582.0,-53.72,50.0,L-SC,10.89,270.0,-0.53,0.49,67.86,XR,NTT,HOTELS
44,JCHAC,2282.00,3.78,-35.09,54.04,-0.01,44831.0,-44847.0,82958.0,16878.26,34.0,M-SC,6.62,233.0,-1.00,0.59,6.18,OX40N,NTT,AC
22,DIXON,18931.72,2.36,-8.80,38.24,26.07,68081.0,-17186.0,178035.0,-55.63,41.0,X-MC,8.33,56.0,-0.25,1.27,10.88,X40N,ATH,IT
74,TITAGARH,1548.00,2.31,-25.02,94.37,45.74,171356.0,-60598.0,181579.0,-0.20,43.0,H-SC,4.89,174.0,-0.35,1.30,18.10,XY24,NTT,ENGINEERING


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.31,-17.69,120.72,81.67,93214.0,-16595.0,77215.0,6938.89,42.0,L-SC,12.97,271.0,-0.18,0.55,49.53,XR,NTT,CERAMICS
14,CAMPUS,393.00,-1.92,-12.40,52.15,33.28,76646.0,-20802.0,146973.0,-29.44,35.0,M-SC,2.62,222.0,-0.27,1.05,13.91,XY24,NTT,FOOTWEAR
67,SONACOMS,806.63,-1.16,-15.79,65.70,39.54,55970.0,-15969.0,85190.0,-34.10,49.0,M-SC,4.64,220.0,-0.29,0.61,20.13,AR,ATH,AUTO
16,CERA,9475.00,-1.12,-32.01,82.23,23.89,98337.0,-56315.0,119588.0,-34.37,28.0,H-SC,10.29,157.0,-0.57,0.85,3.28,OX40N,NTT,CERAMICS
26,GILLETTE,11206.78,-1.01,-4.78,39.07,32.42,94454.0,-12141.0,241755.0,-21.79,37.0,X-SC,9.43,85.0,-0.13,1.72,9.59,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,398.61,0.74,-0.35,22.48,22.05,21948.0,-345.0,97635.0,-14.18,60.0,H-SC,7.19,164.0,-0.02,0.7,41.92,XR,ATH,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.91,-5.41,119.80,107.91,167314.0,-7983.0,139661.0,-22.98,45.0,M-SC,12.66,216.0,-0.05,1.00,4.13,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.79,-11.66,13.22,0.02,20386.0,-20353.0,154206.0,134.93,54.0,M-SC,15.37,234.0,-1.00,1.10,34.13,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.04,-16.77,20.26,0.09,16994.0,-16898.0,83879.0,94.66,42.0,M-SC,2.36,253.0,-0.99,0.60,10.20,OX40N,NTT,DURABLES
66,SIS,528.00,1.77,-24.07,59.71,21.26,50337.0,-26729.0,84303.0,1989.12,50.0,H-SC,4.85,166.0,-0.53,0.60,13.90,OX40N,NTT,MISC
47,KANSAINER,340.00,0.89,-24.74,50.78,13.47,103058.0,-66717.0,202950.0,-69.09,36.0,H-SC,4.76,158.0,-0.65,1.45,3.19,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.11,7.01,60.98,72.27,110572.0,11877.0,181325.0,-8.35,69.0,M-LC,2.21,99.0,0.11,1.29,13.62,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.11,7.01,60.98,72.27,110572.0,11877.0,181325.0,-8.35,69.0,M-LC,2.21,99.0,0.11,1.29,13.62,XR,NTT,IT
50,MASFIN,398.61,0.74,-0.35,22.48,22.05,21948.0,-345.0,97635.0,-14.18,60.0,H-SC,7.19,164.0,-0.02,0.70,41.92,XR,ATH,FINANCE
38,INDIAMART,4810.62,1.55,-2.60,112.24,106.72,134834.0,-3206.0,120130.0,-51.77,39.0,H-SC,2.66,139.0,-0.02,0.86,20.17,AR,ATH,MISC
25,FINCABLES,1641.55,0.91,-5.41,119.80,107.91,167314.0,-7983.0,139661.0,-22.98,45.0,M-SC,12.66,216.0,-0.05,1.00,4.13,OX40N,ATH,CABLES
37,IEX,219.00,-0.31,-3.12,53.40,48.63,103449.0,-6229.0,193725.0,-35.53,51.0,H-SC,14.03,136.0,-0.06,1.38,8.08,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.87,-18.79,91.46,55.47,180691.0,-45723.0,197563.0,-28.03,23.0,X-MC,12.49,64.0,-0.25,1.41,1.23,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.62,-8.89,40.61,28.11,74519.0,-17910.0,183500.0,-34.01,30.0,X-MC,12.67,55.0,-0.24,1.31,0.01,X40,ATH,APPARELS
75,TMPV,600.00,-0.19,-28.92,73.01,22.98,117231.0,-65330.0,160568.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.15,0.99,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,0.96,-11.22,28.03,13.67,63973.0,-28833.0,228230.0,-17.78,31.0,X-LC,2.17,14.0,-0.45,1.63,7.74,XY25,NTT,FMCG
2,ABBOTINDIA,35195.00,-0.50,-6.51,26.28,18.07,43945.0,-11635.0,167220.0,-20.03,32.0,X-MC,5.49,63.0,-0.26,1.19,10.81,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.28,-1.32,26.15,24.48,64989.0,-3333.0,248523.0,-81.09,44.0,X-SC,0.67,86.0,-0.05,1.77,21.51,X40N,NTT,MISC
43,ITC,452.0,0.47,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.69,X40,NTT,FMCG
36,ICICIPRULI,790.0,-0.24,6.90,22.29,30.73,46942.0,13591.0,210596.0,-18.05,63.0,X-MC,2.13,75.0,0.29,1.50,20.55,X40,ATH,INSURANCE
33,HINDUNILVR,2922.0,0.96,-11.22,28.03,13.67,63973.0,-28833.0,228230.0,-17.78,31.0,X-LC,2.17,14.0,-0.45,1.63,7.74,XY25,NTT,FMCG
3,ACC,3906.0,0.24,-25.36,119.93,64.15,212996.0,-60351.0,177600.0,-56.31,35.0,X-SC,2.42,82.0,-0.28,1.27,0.24,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.62,-8.89,40.61,28.11,74519.0,-17910.0,183500.0,-34.01,30.0,X-MC,12.67,55.0,-0.24,1.31,0.01,X40,ATH,APPARELS
3,ACC,3906.00,0.24,-25.36,119.93,64.15,212996.0,-60351.0,177600.0,-56.31,35.0,X-SC,2.42,82.0,-0.28,1.27,0.24,XY24,BTT,CEMENT
75,TMPV,600.00,-0.19,-28.92,73.01,22.98,117231.0,-65330.0,160568.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.15,0.99,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.10,-38.12,118.50,35.20,94347.0,-49052.0,79618.0,4.40,34.0,X-SC,15.99,92.0,-0.52,0.57,1.13,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.87,-18.79,91.46,55.47,180691.0,-45723.0,197563.0,-28.03,23.0,X-MC,12.49,64.0,-0.25,1.41,1.23,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.25,-29.59,102.88,42.84,47087.0,-19237.0,45769.0,-53.72,42.0,X-SC,35.93,83.0,-0.41,0.33,3.55,XY24,NTT,MISC
59,RELAXO,1176.00,-0.16,-47.63,190.80,52.31,145060.0,-69133.0,76027.0,-44.25,42.0,X-SC,6.71,91.0,-0.48,0.54,1.77,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.10,-38.12,118.50,35.20,94347.0,-49052.0,79618.0,4.40,34.0,X-SC,15.99,92.0,-0.52,0.57,1.13,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.82,-4.56,30.01,24.08,36360.0,-5792.0,121158.0,-12.23,33.0,X-SC,5.24,89.0,-0.16,0.86,14.16,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.58,-16.28,71.19,43.32,97075.0,-26512.0,136360.0,-28.06,38.0,X-SC,4.62,90.0,-0.27,0.97,4.70,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.04,-11.42,36.26,20.71,121492.0,-43180.0,335057.0,-23.62,64.0,X-LC,4.66,1.0,-0.36,2.39,11.96,X40,ATH,IT
41,INFY,1972.00,0.45,9.29,22.84,34.24,78101.0,29062.0,341950.0,-14.93,64.0,X-LC,3.58,2.0,0.37,2.44,18.34,X40,NTT,IT
75,TMPV,600.00,-0.19,-28.92,73.01,22.98,117231.0,-65330.0,160568.0,-26.74,30.0,X-LC,6.47,3.0,-0.56,1.15,0.99,XY24,NTT,AUTO
81,VBL,671.64,-0.89,-3.91,41.19,35.67,125010.0,-12345.0,303497.0,-15.30,55.0,X-LC,4.49,4.0,-0.10,2.16,9.40,X40N,ATH,FMCG
43,ITC,452.00,0.47,-1.47,12.44,10.78,29455.0,-3540.0,236778.0,-41.15,46.0,X-LC,1.91,5.0,-0.12,1.69,3.69,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.17,-37.42,112.36,32.89,54284.0,-28894.0,48313.0,-696.43,55.0,L-MC,5.96,259.0,-0.53,0.34,33.05,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.31,-17.69,120.72,81.67,93214.0,-16595.0,77215.0,6938.89,42.0,L-SC,12.97,271.0,-0.18,0.55,49.53,XR,NTT,CERAMICS
50,MASFIN,398.61,0.74,-0.35,22.48,22.05,21948.0,-345.0,97635.0,-14.18,60.0,H-SC,7.19,164.0,-0.02,0.70,41.92,XR,ATH,FINANCE
13,BSOFT,831.70,0.57,-21.65,92.46,50.79,101889.0,-30455.0,110198.0,0.50,68.0,H-SC,6.07,151.0,-0.30,0.79,28.31,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.79,-11.66,13.22,0.02,20386.0,-20353.0,154206.0,134.93,54.0,M-SC,15.37,234.0,-1.00,1.10,34.13,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.57,-21.65,92.46,50.79,101889.0,-30455.0,110198.0,0.50,68.0,H-SC,6.07,151.0,-0.30,0.79,28.31,XR,ATH,IT
84,WIPRO,420.00,0.11,7.01,60.98,72.27,110572.0,11877.0,181325.0,-8.35,69.0,M-LC,2.21,99.0,0.11,1.29,13.62,XR,NTT,IT
41,INFY,1972.00,0.45,9.29,22.84,34.24,78101.0,29062.0,341950.0,-14.93,64.0,X-LC,3.58,2.0,0.37,2.44,18.34,X40,NTT,IT
32,HCLTECH,1922.01,0.62,9.25,14.17,24.73,37453.0,22386.0,264310.0,14.03,66.0,X-LC,6.72,7.0,0.60,1.89,25.90,X40,ATH,IT
1,ABB,7934.00,0.14,-3.03,50.11,45.57,127131.0,-7915.0,253704.0,-37.68,60.0,H-MC,4.58,120.0,-0.06,1.81,7.95,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.31
1,25,44.91
2,50,76.64


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.99
MC    30.90
LC    24.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.81
X40      22.95
X40N     13.00
XR        9.80
XY25      9.07
AR        9.06
OX40N     7.54
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.39
H-SC    23.91
X-LC    20.66
M-SC    11.64
X-SC     8.03
H-MC     4.83
L-SC     1.41
M-MC     1.34
M-LC     1.29
H-LC     1.13
L-LC     1.01
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.30,-7.51,42.09
IT,13.03,-15.16,73.07
FINANCE,9.57,-18.87,70.57
MISC,7.32,-26.41,78.12
ELECTRICAL,6.02,-11.08,51.76
PAINTS,5.87,-12.33,29.02
INSURANCE,4.78,-1.23,35.88
PHARMA,4.11,-1.84,33.89
AUTO,2.83,-32.67,78.66


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3165692.0,21
AR,1301143.0,10
XR,1285460.0,13
X40,1004621.0,14
X40N,860742.0,9
OX40N,735152.0,10
XY25,370066.0,6
SR,282038.0,2
MH,75688.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3625066.0,25
M-SC,1425110.0,15
X-MC,1405357.0,16
X-LC,831791.0,11
X-SC,792368.0,8
H-MC,402270.0,3
L-SC,264088.0,3
M-LC,110572.0,1
H-LC,75081.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1263319.0      6
           AR         891735.0      5
M-SC       XY24       819453.0      6
H-SC       XR         769442.0      7
X-MC       X40        491218.0      7
           XY24       399948.0      3
H-SC       OX40N      342844.0      4
X-MC       X40N       335166.0      4
X-LC       X40        324602.0      5
X-SC       X40N       307124.0      3
M-SC       OX40N      306448.0      5
X-SC       XY24       296443.0      3
H-SC       SR         282038.0      2
X-LC       X40N       218452.0      2
H-MC       AR         208052.0      2
           XY24       194218.0      1
X-LC       XY24       192311.0      2
X-SC       X40        188801.0      2
X-MC       XY25       179025.0      2
L-SC       XR         178228.0      2
M-SC       XR         172934.0      2
           AR         126275.0      2
M-LC       XR         110572.0      1
X-LC       XY25        96426.0      2
L-SC       OX40N       85860.0      1
H-SC       MH          75688.0      1
H-LC       AR          75081.0      1
M-MC       XY25        57328.0      1
L-MC       XR          54284.0      1
L-LC       XY25        37287.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
